In [43]:
import pandas as pd
import os
import os.path as osp
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, auc
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
import random
from tensorflow.keras.models import load_model
import os
import shutil
from typing import List
from PIL import Image

import shutil
import glob
from ultralytics import YOLO


In [ ]:


path_train = '/Users/adelgado/Documents/Personal/Universidad/ML/Proyecto/Data/Train'
path_train_modi = '/Users/adelgado/Documents/Personal/Universidad/ML/Proyecto/Data/Train_modi'
path_test = '/Users/adelgado/Documents/Personal/Universidad/ML/Proyecto/Data/Test'
pathAll = '/Users/adelgado/Documents/Personal/Universidad/ML/Proyecto/Data'

def imagen_90(image_path):
    imagen = Image.open(image_path)
    imagen_rgb = imagen.convert('RGB')
    return imagen_rgb.rotate(90)

def imagen_hor(image_path):
    imagen = Image.open(image_path)
    imagen_rgb = imagen.convert('RGB')
    return imagen_rgb.transpose(Image.FLIP_LEFT_RIGHT)

def imagen_ver(image_path):
    imagen = Image.open(image_path)
    imagen_rgb = imagen.convert('RGB')
    return imagen_rgb.transpose(Image.FLIP_TOP_BOTTOM)

def cambio_carpetas(path, path_destino):
    imagenes = []
    count = 0

    for directorio, _, archivos in os.walk(path):
        for archivo in archivos:
            if archivo.lower().endswith(('.jpg', '.png', '.jpeg')):
                archivo_path = os.path.join(directorio, archivo)
                imagenes.append(archivo_path)
    
    for img_path in imagenes:
        rel_path = os.path.relpath(img_path, path)
        parts = rel_path.split(os.sep)
        
        if len(parts) > 1 and parts[0] == 'Packages':
            continue
            
        if len(parts) > 1:
            class_folder = parts[0]
        else:
            class_folder = 'unknown'

        dest_dir = os.path.join(path_destino, class_folder)
        os.makedirs(dest_dir, exist_ok=True)
        
        filename = os.path.basename(img_path)
        name, ext = os.path.splitext(filename)
        
        try:
            shutil.copy(img_path, os.path.join(dest_dir, filename))
            imagen_90(img_path).save(os.path.join(dest_dir, f"{name}_90{ext}"))
            imagen_hor(img_path).save(os.path.join(dest_dir, f"{name}_hor{ext}"))
            imagen_ver(img_path).save(os.path.join(dest_dir, f"{name}_ver{ext}"))
            
            count += 1
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
    
    print(f"Successfully processed {count} images")
if __name__ == "__main__":
    cambio_carpetas(path_train, path_train_modi)

Successfully processed 209 images


In [7]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')

In [8]:
img_height = 348
img_width = 348
batch_size = 3

In [9]:
train_generator = train_datagen.flow_from_directory(
    path_train_modi,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical') 

Found 836 images belonging to 3 classes.


In [10]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
val_generator = val_datagen.flow_from_directory(
    path_test,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical') 

Found 50 images belonging to 3 classes.


In [22]:
YOLO_VERSION = 'yolov8'
YOLO_DIR = f'/Users/adelgado/Documents/Personal/Universidad/ML/Proyecto/{YOLO_VERSION}'
clases = ["Acero_expuesto","Grietas_Fisuras","Hormiguero"]

In [ ]:
model = YOLO('YOLOv8n-cls.pt')

In [16]:
%pwd

'c:\\Users\\adelgado\\Documents\\Personal\\Universidad\\ML\\Proyecto'

In [26]:
#model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n-cls.pt')

100%|██████████| 5.31M/5.31M [00:00<00:00, 14.6MB/s]


In [33]:
model.train(data = pathAll, epochs=50, imgsz=img_height,batch=16, patience=0, augment=False)

Ultralytics 8.3.140  Python-3.12.10 torch-2.7.0+cpu CPU (Intel Core(TM) i7-10610U 1.80GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/adelgado/Documents/Personal/Universidad/ML/Proyecto/Data, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=348, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=0, perspe

train: Scanning C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train... 836 images, 0 corrupt: 100%|██████████| 836/836 [00:00<?, ?it/s]

train: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train\Acero_expuesto\27986_1683299072418.jpg: corrupt JPEG restored and saved
train: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train\Acero_expuesto\32131_1690637135902.jpg: corrupt JPEG restored and saved
train: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train\Acero_expuesto\32131_1690637136021.jpg: corrupt JPEG restored and saved
train: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train\Acero_expuesto\32131_1690637136032.jpg: corrupt JPEG restored and saved
train: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train\Acero_expuesto\37195_1698936828821.jpg: corrupt JPEG restored and saved
train: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train\Acero_expuesto\38567_1701285628483.jpg: corrupt JPEG restored and saved
train: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train\Acero_expuesto\


c:\Users\adelgado\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val... 50 images, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Grietas_Fisuras\58144_1730459084180.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Grietas_Fisuras\58144_1730459102186.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Hormiguero\52202_1722428841887.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Hormiguero\52202_1722428902223.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Hormiguero\52202_1722428928960.jpg: corrupt JPEG restored and saved
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay

...

                   all       0.74          1



50 epochs completed in 3.907 hours.
Optimizer stripped from runs\classify\train\weights\last.pt, 3.0MB
Optimizer stripped from runs\classify\train\weights\best.pt, 3.0MB

Validating runs\classify\train\weights\best.pt...
Ultralytics 8.3.140  Python-3.12.10 torch-2.7.0+cpu CPU (Intel Core(TM) i7-10610U 1.80GHz)
YOLOv8n-cls summary (fused): 30 layers, 1,438,723 parameters, 0 gradients
train: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train... found 836 images in 3 classes  
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val... found 50 images in 3 classes  
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:12<00:00,  6.46s/it]


                   all        0.8          1
Speed: 0.0ms preprocess, 23.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001DDA52ED820>
curves: []
curves_results: []
fitness: 0.9000000059604645
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.800000011920929, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9000000059604645}
save_dir: WindowsPath('runs/classify/train')
speed: {'preprocess': 0.0014539999392582104, 'inference': 23.630429999975604, 'loss': 7.199996616691351e-05, 'postprocess': 0.000351999988197349}
task: 'classify'
top1: 0.800000011920929
top5: 1.0

In [35]:
m = model.val()

Ultralytics 8.3.140  Python-3.12.10 torch-2.7.0+cpu CPU (Intel Core(TM) i7-10610U 1.80GHz)
train: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train... found 836 images in 3 classes  
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val... found 50 images in 3 classes  
test: None...
val: Fast image access  (ping: 1.21.8 ms, read: 431.0287.6 MB/s, size: 2027.1 KB)


val: Scanning C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val... 50 images, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Grietas_Fisuras\58144_1730459084180.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Grietas_Fisuras\58144_1730459102186.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Hormiguero\52202_1722428841887.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Hormiguero\52202_1722428902223.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Hormiguero\52202_1722428928960.jpg: corrupt JPEG restored and saved



c:\Users\adelgado\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:15<00:00,  3.77s/it]


                   all        0.8          1
Speed: 0.0ms preprocess, 30.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train3


In [54]:
model = YOLO("C:/Users/adelgado/Documents/Personal/Universidad/ML/Proyecto/runs/classify/train/weights/best.pt")
resultados = model("C:/Users/adelgado/Documents/Personal/Universidad/ML/Proyecto/Data/train/Acero_expuesto/10319_WhatsApp Image 2021-11-24 at 6.26.54 PM (1).jpeg")


image 1/1 C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train\Acero_expuesto\10319_WhatsApp Image 2021-11-24 at 6.26.54 PM (1).jpeg: 352x352 Acero_expuesto 1.00, Hormiguero 0.00, Grietas_Fisuras 0.00, 71.8ms
Speed: 39.5ms preprocess, 71.8ms inference, 0.1ms postprocess per image at shape (1, 3, 352, 352)


In [57]:
resultados[0].probs.data.tolist()

[0.9999762773513794, 8.032684517544908e-10, 2.368110654060729e-05]

In [58]:
# Métricas del modelo
metrics = model.val() 
results = model.metrics


Ultralytics 8.3.140  Python-3.12.10 torch-2.7.0+cpu CPU (Intel Core(TM) i7-10610U 1.80GHz)
train: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\train... found 836 images in 3 classes  
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val... found 50 images in 3 classes  
test: None...
val: Fast image access  (ping: 3.94.9 ms, read: 326.4320.3 MB/s, size: 2027.1 KB)


val: Scanning C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val... 50 images, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Grietas_Fisuras\58144_1730459084180.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Grietas_Fisuras\58144_1730459102186.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Hormiguero\52202_1722428841887.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Hormiguero\52202_1722428902223.jpg: corrupt JPEG restored and saved
val: C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\Data\val\Hormiguero\52202_1722428928960.jpg: corrupt JPEG restored and saved



c:\Users\adelgado\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:20<00:00,  5.06s/it]


                   all        0.8          1
Speed: 0.0ms preprocess, 56.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\val2


In [59]:
results

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001DD9042FD40>
curves: []
curves_results: []
fitness: 0.9000000059604645
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.800000011920929, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9000000059604645}
save_dir: WindowsPath('runs/classify/val2')
speed: {'preprocess': 0.002626000059535727, 'inference': 56.49572000002081, 'loss': 0.00015600002370774746, 'postprocess': 0.0006580000626854599}
task: 'classify'
top1: 0.800000011920929
top5: 1.0

In [73]:
model.export(format='torchscript')

Ultralytics 8.3.140  Python-3.12.10 torch-2.7.0+cpu CPU (Intel Core(TM) i7-10610U 1.80GHz)

PyTorch: starting from 'C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\runs\classify\train\weights\best.pt' with input shape (1, 3, 352, 352) BCHW and output shape(s) (1, 3) (2.8 MB)

TorchScript: starting export with torch 2.7.0+cpu...
TorchScript: export success  1.7s, saved as 'C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\runs\classify\train\weights\best.torchscript' (5.6 MB)

Export complete (1.9s)
Results saved to C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\runs\classify\train\weights
Predict:         yolo predict task=classify model=C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\runs\classify\train\weights\best.torchscript imgsz=352  
Validate:        yolo val task=classify model=C:\Users\adelgado\Documents\Personal\Universidad\ML\Proyecto\runs\classify\train\weights\best.torchscript imgsz=352 data=/Users/adelgado/Documents/Perso

'C:\\Users\\adelgado\\Documents\\Personal\\Universidad\\ML\\Proyecto\\runs\\classify\\train\\weights\\best.torchscript'

In [75]:
model.tune(data=pathAll, epochs=30, iterations=30, optimizer="AdamW", plots=True, save=True, val=True)

Tuner: Initialized Tuner instance with 'tune_dir=runs\classify\tune2'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/30 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'cutmix': 0.0, 'copy_paste': 0.0}
ERROR training failure for hyperparameter tuning iteration 1
Command '['c:\\Users\\adelgado\\AppData\\Local\\Programs\\Python\\Python312\\python.exe', '-m', 'ultralytics.cfg.__init__', 'train', 'task=classify', 'mode=train', 'model=C:/Users/adelgado/Documents/Personal/Universidad/ML/Proyecto/runs/classify/train/weights/best.pt', 'data=/Users/adelgado/Documents/Personal/Universidad/ML/Proyecto/Data', 'ep